In [2]:
import pickle
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path

In [2]:
tyear = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [3]:
stock = []
for i in range(len(tyear)):
    fname = f'a_share_market_{2011 + i}.pkl'
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    data = data[['trade_dt', 'windcode','open', 'high', 'low', 'close', 'volume']]
    stock.append(data)
    print(data['trade_dt'].nunique(), ' ', data['windcode'].nunique(), end='  *  ')

244   2323  *  243   2475  *  238   2474  *  245   2593  *  244   2815  *  244   3035  *  244   3472  *  243   3572  *  244   3771  *  243   4155  *  243   4707  *  242   5115  *  242   5382  *  153   5392  *  

In [4]:
common_windcodes = set.intersection(*(set(df['windcode']) for df in stock))
len(common_windcodes)

2183

In [5]:
stock = [df[df['windcode'].isin(common_windcodes)] for df in stock]
stock = pd.concat(stock, ignore_index=True)
stock = stock[stock['close'] != 0.0]
stock

,trade_dt,windcode,open,high,low,close,volume
0,20110104,000001.SZ,15.82,16.18,15.65,16.02,358840.61
1,20110105,000001.SZ,15.99,16.13,15.91,15.93,237462.63
2,20110106,000001.SZ,15.93,16.08,15.70,15.81,206531.27
3,20110107,000001.SZ,15.80,16.78,15.70,16.41,770482.21
4,20110110,000001.SZ,16.23,16.40,15.90,15.98,361872.13
...,...,...,...,...,...,...,...
7200237,20240813,601999.SH,5.37,5.38,5.24,5.30,22585.00
7200238,20240814,601999.SH,5.31,5.36,5.30,5.32,25552.00
7200239,20240815,601999.SH,5.32,5.45,5.26,5.42,35806.00
7200240,20240816,601999.SH,5.38,5.42,5.32,5.32,22831.00


In [6]:
grouped_df = stock.groupby('windcode')
l = stock['trade_dt'].nunique()
def filter_groups(group):
    return len(group) == l
stock = grouped_df.filter(filter_groups)
stock

,trade_dt,windcode,open,high,low,close,volume
0,20110104,000001.SZ,15.82,16.18,15.65,16.02,358840.61
1,20110105,000001.SZ,15.99,16.13,15.91,15.93,237462.63
2,20110106,000001.SZ,15.93,16.08,15.70,15.81,206531.27
3,20110107,000001.SZ,15.80,16.78,15.70,16.41,770482.21
4,20110110,000001.SZ,16.23,16.40,15.90,15.98,361872.13
...,...,...,...,...,...,...,...
7200237,20240813,601999.SH,5.37,5.38,5.24,5.30,22585.00
7200238,20240814,601999.SH,5.31,5.36,5.30,5.32,25552.00
7200239,20240815,601999.SH,5.32,5.45,5.26,5.42,35806.00
7200240,20240816,601999.SH,5.38,5.42,5.32,5.32,22831.00


In [7]:
stock['windcode'].nunique()

1805

In [8]:
stock['trade_dt'].nunique()

3312

In [9]:
stock_list = []
with open(f'hs300A.txt') as f:
    for s in f:
        stock_list.append(s.strip())

print(len(stock_list))

559


In [10]:
unique_windcodes = stock['windcode'].unique()
is_subset = set(stock_list).issubset(set(unique_windcodes))
is_subset

False

In [11]:
stock = stock[stock['windcode'].isin(stock_list)]

In [12]:
stock['windcode'].nunique()

426

In [13]:
unique_windcodes = stock['windcode'].unique()

with open('stock_list.txt', 'w') as f:
    for windcode in unique_windcodes:
        f.write(f"{windcode}\n")

In [14]:
stock['trade_dt'].nunique()

3312

In [15]:
stock.rename(columns={'windcode':'instrument','trade_dt':'datetime'},inplace=True)
stock = stock.sort_values(by=['instrument', 'datetime'])
# stock['label'] = stock.groupby('instrument')['close'].pct_change().shift(-1)
stock['label'] = stock.groupby('instrument')['close'].apply(lambda x: x.shift(-1) / x - 1)
# stock['label'] = stock['label'].fillna(0)
stock

/tmp/ipykernel_2447349/3414935398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock.rename(columns={'windcode':'instrument','trade_dt':'datetime'},inplace=True)
/tmp/ipykernel_2447349/3414935398.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stock['label'] = stock.groupby('instrument')['close'].apply(lambda x: x.shift(-1) / x - 1)


,datetime,instrument,open,high,low,close,volume,label
0,20110104,000001.SZ,15.82,16.18,15.65,16.02,358840.61,-0.005618
1,20110105,000001.SZ,15.99,16.13,15.91,15.93,237462.63,-0.007533
2,20110106,000001.SZ,15.93,16.08,15.70,15.81,206531.27,0.037951
3,20110107,000001.SZ,15.80,16.78,15.70,16.41,770482.21,-0.026204
4,20110110,000001.SZ,16.23,16.40,15.90,15.98,361872.13,0.013141
...,...,...,...,...,...,...,...,...
7200084,20240813,601998.SH,6.11,6.22,6.09,6.22,450315.34,0.016077
7200085,20240814,601998.SH,6.22,6.33,6.21,6.32,370990.54,0.017405
7200086,20240815,601998.SH,6.32,6.45,6.28,6.43,508701.93,0.012442
7200087,20240816,601998.SH,6.42,6.53,6.40,6.51,414296.06,0.015361


In [16]:
stock[stock.isna().any(axis=1)]

,datetime,instrument,open,high,low,close,volume,label
6867528,20240819,000001.SZ,10.13,10.30,10.12,10.29,1482985.26,NaN
6867681,20240819,000002.SZ,6.77,6.88,6.74,6.80,749189.01,NaN
6868369,20240819,000008.SZ,2.17,2.23,2.11,2.13,630449.54,NaN
6868522,20240819,000009.SZ,7.87,7.98,7.82,7.89,97851.31,NaN
6868981,20240819,000012.SZ,5.12,5.17,5.11,5.13,73451.92,NaN
...,...,...,...,...,...,...,...,...
7199170,20240819,601958.SH,9.83,10.05,9.83,9.93,89009.19,NaN
7199323,20240819,601988.SH,4.81,4.93,4.79,4.92,1929694.69,NaN
7199476,20240819,601989.SH,5.45,5.55,5.33,5.53,1872029.28,NaN
7199629,20240819,601991.SH,2.91,2.92,2.88,2.90,404455.99,NaN


In [17]:
stock = stock.dropna()
stock

,datetime,instrument,open,high,low,close,volume,label
0,20110104,000001.SZ,15.82,16.18,15.65,16.02,358840.61,-0.005618
1,20110105,000001.SZ,15.99,16.13,15.91,15.93,237462.63,-0.007533
2,20110106,000001.SZ,15.93,16.08,15.70,15.81,206531.27,0.037951
3,20110107,000001.SZ,15.80,16.78,15.70,16.41,770482.21,-0.026204
4,20110110,000001.SZ,16.23,16.40,15.90,15.98,361872.13,0.013141
...,...,...,...,...,...,...,...,...
7200083,20240812,601998.SH,6.12,6.14,6.09,6.11,241954.68,0.018003
7200084,20240813,601998.SH,6.11,6.22,6.09,6.22,450315.34,0.016077
7200085,20240814,601998.SH,6.22,6.33,6.21,6.32,370990.54,0.017405
7200086,20240815,601998.SH,6.32,6.45,6.28,6.43,508701.93,0.012442


In [18]:
stock['datetime'].nunique()

3311

In [19]:
stock['instrument'].nunique()

426

In [20]:
np.isinf(stock['label']).any()

False

In [21]:
stock['datetime'] = pd.to_datetime(stock['datetime'], format='%Y%m%d')
stock

/tmp/ipykernel_2447349/425243358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['datetime'] = pd.to_datetime(stock['datetime'], format='%Y%m%d')


,datetime,instrument,open,high,low,close,volume,label
0,2011-01-04,000001.SZ,15.82,16.18,15.65,16.02,358840.61,-0.005618
1,2011-01-05,000001.SZ,15.99,16.13,15.91,15.93,237462.63,-0.007533
2,2011-01-06,000001.SZ,15.93,16.08,15.70,15.81,206531.27,0.037951
3,2011-01-07,000001.SZ,15.80,16.78,15.70,16.41,770482.21,-0.026204
4,2011-01-10,000001.SZ,16.23,16.40,15.90,15.98,361872.13,0.013141
...,...,...,...,...,...,...,...,...
7200083,2024-08-12,601998.SH,6.12,6.14,6.09,6.11,241954.68,0.018003
7200084,2024-08-13,601998.SH,6.11,6.22,6.09,6.22,450315.34,0.016077
7200085,2024-08-14,601998.SH,6.22,6.33,6.21,6.32,370990.54,0.017405
7200086,2024-08-15,601998.SH,6.32,6.45,6.28,6.43,508701.93,0.012442


In [3]:
# stock.to_csv("HS300A.csv", index=None)
stock = pd.read_csv("HS300A.csv")

In [4]:
'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# columns_to_scale = ['close', 'open', 'high', 'low', 'turnover', 'volume']
columns_to_scale = ['close', 'open', 'high', 'low', 'volume']
stock[columns_to_scale] = scaler.fit_transform(stock[columns_to_scale])
stock
'''
def process_daily_df_std(df, feature_cols):
    df = df.copy()
    for c in feature_cols:
        df[c] = filter_extreme_3sigma(df[c])
        df[c] = standardize_zscore(df[c])
    return df


def filter_extreme_3sigma(series, n=3):
    mean = series.mean()
    std = series.std()
    max_range = mean + n * std
    min_range = mean - n * std
    return np.clip(series, min_range, max_range)


def standardize_zscore(series):
    std = series.std()
    mean = series.mean()
    return (series - mean) / std

def process_features(df_features, feature_cols):
    df_features_grouped = df_features.groupby("datetime")
    res = []
    for dt in df_features_grouped.groups:
        df = df_features_grouped.get_group(dt)
        processed_df = process_daily_df_std(df, feature_cols)
        res.append(processed_df)
    df_features = pd.concat(res)
    df_features = df_features.dropna(subset=feature_cols)
    return df_features

columns_to_scale = ['close', 'open', 'high', 'low', 'volume']
stock = process_features(stock, columns_to_scale)
stock

,datetime,instrument,open,high,low,close,volume,label
0,2011-01-04,000001.SZ,-0.458603,-0.458407,-0.455212,-0.456117,0.655258,-0.005618
3311,2011-01-04,000002.SZ,-0.788557,-0.766833,-0.785551,-0.770316,3.732129,0.003409
6622,2011-01-04,000008.SZ,-0.629480,-0.632325,-0.625279,-0.628013,-0.735461,0.000829
9933,2011-01-04,000009.SZ,-0.416648,-0.414284,-0.414699,-0.416081,-0.081673,-0.020071
13244,2011-01-04,000012.SZ,-0.280733,-0.282346,-0.288262,-0.281610,0.763447,-0.013480
...,...,...,...,...,...,...,...,...
1397241,2024-08-16,601958.SH,-0.193861,-0.195597,-0.194264,-0.194523,-0.549703,0.008122
1400552,2024-08-16,601988.SH,-0.425933,-0.425460,-0.423026,-0.421036,2.842037,0.022869
1403863,2024-08-16,601989.SH,-0.385983,-0.386182,-0.383083,-0.383284,1.050177,-0.021239
1407174,2024-08-16,601991.SH,-0.508079,-0.509371,-0.507450,-0.506428,0.436644,-0.003436


In [11]:
feaData = stock.drop(columns=['label'])
feaData['datetime'] = pd.to_datetime(feaData['datetime']) 

feaData = feaData.sort_values(by=['datetime', 'instrument']).set_index(['datetime', 'instrument'])



In [12]:

feaData.to_pickle('HS1024fea.pkl')

In [ ]:
labData = stock.drop(columns=columns_to_scale)
labData['datetime'] = pd.to_datetime(labData['datetime']) 

np.isinf(labData['label']).any()

False

In [7]:
labData = labData.sort_values(by=['datetime', 'instrument']).set_index(['datetime', 'instrument'])
labData

label
datetime   instrument          
2011-01-04 000001.SZ  -0.005618
           000002.SZ   0.003409
           000008.SZ   0.000829
           000009.SZ  -0.020071
           000012.SZ  -0.013480
...                         ...
2024-08-16 601958.SH   0.008122
           601988.SH   0.022869
           601989.SH  -0.021239
           601991.SH  -0.003436
           601998.SH   0.015361

[1410486 rows x 1 columns]

In [8]:
labData.to_pickle('HS1024lab.pkl')